In [ ]:
import xgboost as xgb
import pandas as pd
#import numerapi
import sklearn.linear_model

import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle
import time

# gradient boosting for classification in scikit-learn
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.externals import joblib
from sklearn.utils import class_weight

from datetime import date, timedelta
import os

In [ ]:
def oneHotEncodeData(targets):
    j=0
    Y_val = np.zeros((targets.shape[0], 5))
    for j in range(targets.shape[0]):
        if targets[j] == 0:
            Y_val[j, 0] = 1
        elif targets[j] == 0.25:
            Y_val[j, 1] = 1
        elif targets[j] == 0.5:
            Y_val[j, 2] = 1
        elif targets[j] == 0.75:
            Y_val[j, 3] = 1
        elif targets[j] == 1.0:
            Y_val[j, 4] = 1
        else:
            print("something went wrong, new class", targets[j])
    return Y_val

def calcMaxAccuracy(y_true, y_pred):
    num_correct_classified = np.argmax(y_true,axis = 1) == np.argmax(y_pred, axis = 1)
    num = np.sum(num_correct_classified.astype(int), axis = -1)
    retVal = num / y_true.shape[0]
    return retVal

def calcMaxAccuracy2(Y_true, Y_pred):
    corVal = 0
    allTrainSamples = len(Y_pred)
    for i in range(allTrainSamples):
        if Y_pred[i] == Y_true[i]:
            corVal +=1
    accuracy = corVal / allTrainSamples
    return accuracy

In [ ]:
start_date = date(2021,1, 31)
tournament_data = pd.read_csv("data/numerai_datasets_28.03.21/numerai_tournament_data.csv")
feature_cols    = tournament_data.columns[tournament_data.columns.str.startswith('feature')]
validation_data = tournament_data.loc[tournament_data.data_type == 'validation']
X_val = validation_data[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()
Y_val = validation_data.target.to_numpy()
#Y_val = oneHotEncodeData(Y_val)
#D_test = xgb.DMatrix(X_val, label=Y_val)

In [ ]:
#start_date = date(2021,3, 21)
path = "data/numerai_datasets_"
test_date = start_date
test_path = path + test_date.strftime("%d.%m.%y")
delta = timedelta(days=7)

steps = 5
counter = 1
epochs = 1

epoch = 0
eta = 0.5
gamma = 0.1
for epoch in range(epochs):
    test_date = start_date
    test_path = path + test_date.strftime("%d.%m.%y")
    print("epoch =", epoch)
    
    model = XGBClassifier(
        #base_score=0.0, booster='gbtree', colsample_bylevel=1,
    #    colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
    #    max_delta_step=0.0, max_depth=8, min_child_weight=1, missing=None,
        n_estimators=steps, #n_jobs=10, nthread=None,
        objective='multi:softprob', #random_state=0, 
    #    reg_alpha=0.05,
    #    reg_lambda=1.0, scale_pos_weight=1, seed=None, silent=None,
    #    subsample=0.8, verbosity=0
    #    eta = eta,
    #    subsample=0.5, colsample_bytree=0.5,
        max_depth = 4,
    #    gamma = gamma
       ) 
#     eta = eta * 0.9
#     gamma = gamma*1.5
    while os.path.exists(test_path):
        path_to_csv = test_path + "/numerai_training_data.csv" 
        tic = time.time()
        training_data = pd.read_csv(str(path_to_csv))
        toc = time.time()
        print("loaded data ",toc-tic, "date = ", test_path)
    
        X_train = training_data[feature_cols].reset_index().drop(['index'], axis = 1).to_numpy()
        Y_train = training_data.target.to_numpy()
    
        classes = np.array([0, 0.25, 0.5, 0.75, 1])
        class_weights = class_weight.compute_class_weight('balanced', np.array([0, 0.25, 0.5, 0.75, 1]), Y_train)

        w_array = np.ones(Y_train.shape[0], dtype = 'float')
        for i, val in enumerate(Y_train):
            index = np.where(classes == val)
            w_array[i] = class_weights[index]
    
        #D_train = xgb.DMatrix(X_train, label=Y_train, weight = w_array)
        evalset = [(X_train, Y_train), (X_val,Y_val)]
        #watchlist  = [ (D_train,'train'),(D_test,'eval')]
        earlyStopping = 10
    
        if counter == 0:
            model.fit(X_train, Y_train,  eval_set=evalset, sample_weight=w_array, early_stopping_rounds=earlyStopping)
        else:
            model.fit(X_train, Y_train,  eval_set=evalset, sample_weight=w_array, early_stopping_rounds=earlyStopping, xgb_model='model_kFold.model')
        model.save_model('model_kFold.model')
    
#         if counter == 0:
#             model.fit(X_train, Y_train,  eval_set=evalset, early_stopping_rounds=earlyStopping)
#         else:
#             model.fit(X_train, Y_train,  eval_set=evalset, early_stopping_rounds=earlyStopping, xgb_model='model_kFold_noWeight.model')
#         model.save_model('model_kFold_noWeight.model')
   
    
        Y_train_pred = model.predict(X_train)
        accTrain     = calcMaxAccuracy2(Y_train, Y_train_pred)
        print("train accuracy = ", accTrain)


        y_pred = model.predict(X_val)
        accVal = calcMaxAccuracy2(Y_val, y_pred)
        print("validation accuracy = ", accVal)
    
        
    
        test_date = test_date + delta
        test_path = path + test_date.strftime("%d.%m.%y")
        counter += 1
    

In [ ]:

Y_train_pred = model.predict(X_train)
corVal = 0
allTrainSamples = len(Y_train_pred)
for i in range(allTrainSamples):
    if Y_train_pred[i] == Y_train[i]:
        corVal +=1
accuracy = corVal / allTrainSamples
print("train accuracy = ", accuracy)


y_pred = model.predict(X_val)
corVal = 0
allTrainSamples = len(y_pred)
for i in range(allTrainSamples):
    if y_pred[i] == Y_val[i]:
        corVal +=1
accuracy = corVal / allTrainSamples
print("validation accuracy = ", accuracy)

In [ ]:
predictions_df = validation_data["id"].to_frame()
predictions_df["pred"] = y_pred
predictions_df.pred.hist(bins = 10)

In [ ]:
predictions_df = training_data["id"].to_frame()
predictions_df["pred"] = Y_train_pred
predictions_df.pred.hist(bins = 10)